In [ ]:
# Developed by: Aadhavan A P
# Project: Vehicle and Pedestrian Tracking using OpenCV
# Description: Detects moving vehicles and humans using Haar Cascade classifiers

import cv2
import os

# Use absolute path for video on Mac
video_path = '/Users/aadhavanap/Downloads/vehicle-pedestrian-detection-opencv-master/Dataset (Sample Videos)/WhatsApp Video 2025-07-08 at 14.03.20.mp4'

# Get the directory where the current script is located
script_dir = os.path.dirname(os.path.abspath(''))

# Construct paths relative to the script's directory for classifiers
car_tracker_file = os.path.join(script_dir, 'haarcascades', 'haarcascade_car.xml')
pedestrian_tracker_file = os.path.join(script_dir, 'haarcascades', 'haarcascade_fullbody.xml')

# Debug prints to make sure paths are correct
print("Video path exists:", os.path.exists(video_path))
print("Car path exists:", os.path.exists(car_tracker_file))
print("Pedestrian path exists:", os.path.exists(pedestrian_tracker_file))

# Load video
video = cv2.VideoCapture(video_path)

# Step 3: Create classifiers
car_tracker = cv2.CascadeClassifier(car_tracker_file)
pedestrian_tracker = cv2.CascadeClassifier(pedestrian_tracker_file)

# Check if classifiers loaded successfully
if car_tracker.empty():
    print(f"Error: Could not load car cascade classifier from {car_tracker_file}")
    exit()
if pedestrian_tracker.empty():
    print(f"Error: Could not load pedestrian cascade classifier from {pedestrian_tracker_file}")
    exit()

# Check if video opened successfully
if not video.isOpened():
    print(f"Error: Could not open video from {video_path}")
    exit()

# Step 4: Read and process frames
while True:
    (read_successful, frame) = video.read()
    
    if not read_successful:
        print("End of video stream or error reading frame.")
        break

    greyscaled_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply histogram equalization for better contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    greyscaled_frame = clahe.apply(greyscaled_frame)
    
    # Apply Gaussian blur to reduce noise
    greyscaled_frame = cv2.GaussianBlur(greyscaled_frame, (3, 3), 0)

    # Optimized parameters for better accuracy
    # Car detection with stricter parameters
    cars = car_tracker.detectMultiScale(
        greyscaled_frame, 
        scaleFactor=1.05,        # Smaller scale factor for better detection
        minNeighbors=8,          # Higher min neighbors to reduce false positives
        minSize=(30, 30),        # Minimum car size
        maxSize=(300, 300),      # Maximum car size
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    
    # Pedestrian detection with optimized parameters
    pedestrians = pedestrian_tracker.detectMultiScale(
        greyscaled_frame, 
        scaleFactor=1.03,        # Even smaller scale factor for pedestrians
        minNeighbors=6,          # Higher min neighbors
        minSize=(20, 60),        # Minimum pedestrian size (width, height)
        maxSize=(100, 300),      # Maximum pedestrian size
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Draw rectangles around detected cars with confidence filtering
    for (x, y, w, h) in cars:
        # Filter out very small or very large detections
        if w > 25 and h > 25 and w < 350 and h < 350:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 3)
            cv2.putText(frame, 'VEHICLE', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            # Add confidence indicator
            cv2.putText(frame, f'W:{w} H:{h}', (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)

    # Draw rectangles around detected pedestrians with confidence filtering
    for (x, y, w, h) in pedestrians:
        # Filter pedestrians based on aspect ratio (height should be > width)
        if w > 15 and h > 50 and h > w * 1.5 and w < 120 and h < 350:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 3)
            cv2.putText(frame, 'HUMAN', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            # Add confidence indicator
            cv2.putText(frame, f'W:{w} H:{h}', (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 255), 1)

    # Display output frame
    cv2.imshow('Vehicle and Pedestrian Detector - Aadhavan A P', frame)

    key = cv2.waitKey(1)

    # Press 'q' or 'Q' to exit
    if key == ord('q') or key == ord('Q'):
        break

# Release resources
video.release()
cv2.destroyAllWindows()